In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
import seaborn as sns

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv("train.csv")

In [3]:
x = df.loc[:,["OverallQual","YearBuilt","YearRemodAdd","MasVnrArea","TotalBsmtSF","1stFlrSF","GrLivArea","FullBath","TotRmsAbvGrd","Fireplaces","GarageYrBlt","GarageCars","GarageArea"]]

In [4]:
x.isnull().sum()

OverallQual      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       8
TotalBsmtSF      0
1stFlrSF         0
GrLivArea        0
FullBath         0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt     81
GarageCars       0
GarageArea       0
dtype: int64

In [5]:
x['GarageYrBlt'].describe()
#차고 건설 연도

count    1379.000000
mean     1978.506164
std        24.689725
min      1900.000000
25%      1961.000000
50%      1980.000000
75%      2002.000000
max      2010.000000
Name: GarageYrBlt, dtype: float64

In [6]:
x=x.drop("GarageYrBlt",axis=1)

In [7]:
x['MasVnrArea'].describe()

count    1452.000000
mean      103.685262
std       181.066207
min         0.000000
25%         0.000000
50%         0.000000
75%       166.000000
max      1600.000000
Name: MasVnrArea, dtype: float64

In [8]:
x.loc[df.isnull()["MasVnrArea"],:] = x.loc[df.isnull()["MasVnrArea"],:].fillna(value=x["MasVnrArea"].mean())
# 평균으로 결측값을 채움

In [9]:
x.isnull().sum()

OverallQual     0
YearBuilt       0
YearRemodAdd    0
MasVnrArea      0
TotalBsmtSF     0
1stFlrSF        0
GrLivArea       0
FullBath        0
TotRmsAbvGrd    0
Fireplaces      0
GarageCars      0
GarageArea      0
dtype: int64

In [10]:
x.head()

,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,TotalBsmtSF,1stFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea
0,7,2003,2003,196.0,856,856,1710,2,8,0,2,548
1,6,1976,1976,0.0,1262,1262,1262,2,6,1,2,460
2,7,2001,2002,162.0,920,920,1786,2,6,1,2,608
3,7,1915,1970,0.0,756,961,1717,1,7,1,3,642
4,8,2000,2000,350.0,1145,1145,2198,2,9,1,3,836


In [11]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 12 columns):
OverallQual     1460 non-null int64
YearBuilt       1460 non-null int64
YearRemodAdd    1460 non-null int64
MasVnrArea      1460 non-null float64
TotalBsmtSF     1460 non-null int64
1stFlrSF        1460 non-null int64
GrLivArea       1460 non-null int64
FullBath        1460 non-null int64
TotRmsAbvGrd    1460 non-null int64
Fireplaces      1460 non-null int64
GarageCars      1460 non-null int64
GarageArea      1460 non-null int64
dtypes: float64(1), int64(11)
memory usage: 137.0 KB


In [12]:
x = x.values
# array로 변경

In [13]:
x = x.astype(np.float32)
x

array([[7.000e+00, 2.003e+03, 2.003e+03, ..., 0.000e+00, 2.000e+00,
        5.480e+02],
       [6.000e+00, 1.976e+03, 1.976e+03, ..., 1.000e+00, 2.000e+00,
        4.600e+02],
       [7.000e+00, 2.001e+03, 2.002e+03, ..., 1.000e+00, 2.000e+00,
        6.080e+02],
       ...,
       [7.000e+00, 1.941e+03, 2.006e+03, ..., 2.000e+00, 1.000e+00,
        2.520e+02],
       [5.000e+00, 1.950e+03, 1.996e+03, ..., 0.000e+00, 1.000e+00,
        2.400e+02],
       [5.000e+00, 1.965e+03, 1.965e+03, ..., 0.000e+00, 1.000e+00,
        2.760e+02]], dtype=float32)

In [14]:
len(x)

1460

In [15]:
y_data = df.iloc[:,-1]
y_data.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [16]:
y_data.isnull().sum()

0

In [17]:
x_data= x 

In [18]:
y_data = y_data.values

In [19]:
y_data

array([208500, 181500, 223500, ..., 266500, 142125, 147500], dtype=int64)

In [20]:
y_data = y_data.astype(np.float32)

In [21]:
print(x_data.shape)
print(y_data.shape)

(1460, 12)
(1460,)


In [22]:
y_data=y_data.reshape(-1,1)

In [23]:
print(y_data.shape)

(1460, 1)


In [24]:
X = tf.placeholder(tf.float32, shape=[None,12])
Y = tf.placeholder(tf.float32, shape=[None,1])


W = tf.Variable(tf.random_normal([12,1],name='weight'))
b = tf.Variable(tf.random_normal([1],name='bias'))

hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis-Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-10)
train = optimizer.minimize(cost)

In [25]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [26]:
for step in range(200001):
    cost_val, hy_val, _ = sess.run(
        [cost,hypothesis,train],
        feed_dict={X:x_data, Y:y_data})
    if step % 50000 == 0:
        print(step,"Cost : ",cost_val,
             "\nPreadiction\n",hy_val)

0 Cost :  41262020000.0 
Preadiction
 [[-5753.4556]
 [-5769.6133]
 [-5932.1597]
 ...
 [-6327.1196]
 [-5305.5947]
 [-5580.2627]]
50000 Cost :  2246455000.0 
Preadiction
 [[183381.17]
 [172721.03]
 [193855.75]
 ...
 [228434.17]
 [140581.27]
 [165146.56]]
100000 Cost :  2149606700.0 
Preadiction
 [[188042.88]
 [169957.4 ]
 [199016.7 ]
 ...
 [226185.45]
 [134733.53]
 [159397.94]]
150000 Cost :  2104909300.0 
Preadiction
 [[191137.03]
 [168876.78]
 [202225.78]
 ...
 [222548.75]
 [132092.61]
 [156226.69]]
200000 Cost :  2082198700.0 
Preadiction
 [[193203.6 ]
 [168297.53]
 [204367.73]
 ...
 [219633.05]
 [130407.56]
 [154146.61]]


---

In [27]:
test = pd.read_csv("test.csv")

In [28]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [29]:
test.shape

(1459, 80)

In [30]:
x = test.loc[:,["OverallQual","YearBuilt","YearRemodAdd","MasVnrArea","TotalBsmtSF","1stFlrSF","GrLivArea","FullBath","TotRmsAbvGrd","Fireplaces","GarageCars","GarageArea"]]

In [31]:
x.isnull().sum()

OverallQual      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea      15
TotalBsmtSF      1
1stFlrSF         0
GrLivArea        0
FullBath         0
TotRmsAbvGrd     0
Fireplaces       0
GarageCars       1
GarageArea       1
dtype: int64

In [32]:
x.loc[test.isnull()["MasVnrArea"],:] = x.loc[test.isnull()["MasVnrArea"],:].fillna(value=x["MasVnrArea"].mean())

In [33]:
x['TotalBsmtSF'].describe()

count    1458.000000
mean     1046.117970
std       442.898624
min         0.000000
25%       784.000000
50%       988.000000
75%      1305.000000
max      5095.000000
Name: TotalBsmtSF, dtype: float64

In [34]:
x.loc[test.isnull()["TotalBsmtSF"],:] 

,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,TotalBsmtSF,1stFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea
660,4,1946,1950,0.0,NaN,896,896,1,4,0,1.0,280.0


In [35]:
x.loc[test.isnull()["TotalBsmtSF"],:] = x.loc[test.isnull()["TotalBsmtSF"],:].fillna(value=x["TotalBsmtSF"].mean())

In [36]:
x.loc[test.isnull()["GarageCars"],:]

,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,TotalBsmtSF,1stFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea
1116,5,1923,1999,0.0,859.0,942,1828,2,6,0,NaN,NaN


In [37]:
x['GarageCars'].describe()

count    1458.000000
mean        1.766118
std         0.775945
min         0.000000
25%         1.000000
50%         2.000000
75%         2.000000
max         5.000000
Name: GarageCars, dtype: float64

In [38]:
x['GarageArea'].describe()

count    1458.000000
mean      472.768861
std       217.048611
min         0.000000
25%       318.000000
50%       480.000000
75%       576.000000
max      1488.000000
Name: GarageArea, dtype: float64

In [39]:
x.loc[test.isnull()["GarageCars"],:] = x.loc[test.isnull()["GarageCars"],:].fillna(value=x["GarageCars"].mean())
x.loc[test.isnull()["GarageArea"],:] = x.loc[test.isnull()["GarageArea"],:].fillna(value=x["GarageArea"].mean())

In [40]:
x.isnull().sum()

OverallQual     0
YearBuilt       0
YearRemodAdd    0
MasVnrArea      0
TotalBsmtSF     0
1stFlrSF        0
GrLivArea       0
FullBath        0
TotRmsAbvGrd    0
Fireplaces      0
GarageCars      0
GarageArea      0
dtype: int64

In [41]:
x.shape

(1459, 12)

In [48]:
sess.run(hypothesis, feed_dict={X:x})

array([[144756.33],
       [170298.44],
       [177643.39],
       ...,
       [172193.28],
       [ 97577.04],
       [222553.16]], dtype=float32)

In [43]:
sample = pd.read_csv("sample_submission.csv")

In [44]:
sample.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [45]:
sample['SalePrice']=sess.run(hypothesis, feed_dict={X:x})

In [46]:
sample.head()

,Id,SalePrice
0,1461,144756.328125
1,1462,170298.437500
2,1463,177643.390625
3,1464,175897.859375
4,1465,173679.093750


In [199]:
sample.to_csv("sample2.csv",index=False)

score : 0.23053